In [3]:
import numpy as np # --- Process Data ---





def scrapetle():
    import urllib2

    data = urllib2.urlopen("http://celestrak.com/NORAD/elements/stations.txt").read(168) # read only 20 000 chars
    lines = data.split("\n") # then split it into lines

    #for line in lines:
    #    print line
    return (lines)

def scrapetle2():
    import requests
    from bs4 import BeautifulSoup
    url = 'http://celestrak.com/NORAD/elements/stations.txt'
    response = requests.get(url)
    html = response.content
    soup = BeautifulSoup(html, "lxml")
    mytle=(soup.text[:168])
    print (mytle)
    print ("was printing tle")
    lines = mytle.splitlines
    dir (mytle)
    print ("now printing line 1")

    print (lines(0))
    return (lines)


def getorbit(mydatestr,npoints):
    npoints=npoints
    lonarr=[0] * npoints
    latarr=[0] * npoints
    from datetime import datetime, timedelta
    import sys, os
    import pytz
    from dateutil.parser import parse
    import ephem
    import math
    mydate=parse(mydatestr)
    
    lines=gettlefromdb(mydate, 1)
    iss = ephem.readtle(lines[0], lines[1], lines[2])

    for i in range(0,npoints):
        one_hour = timedelta(hours=1)
        # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
        
        orbitinminutes=24*60/iss._n
        minint = timedelta(minutes=orbitinminutes/npoints)
        dateinc=mydate +i*minint
        home = ephem.Observer()
        home.date = dateinc
        
        iss.compute(home)
        dist=2
        lonarr[i]=math.degrees(iss.sublong)
        latarr[i]=math.degrees(iss.sublat)
    return (lonarr, latarr)



def readvaisala (fname):
    data_file = open(fname)
    lats, lons = [], []
    magnitudes = []
    timestrings = []
    datestrings = []
    for index, line in enumerate(data_file.readlines()):
        if index > 0:
            datestrings.append((line.split(' ')[0]))
            timestrings.append((line.split(' ')[1]))
            lats.append(float(line.split(' ')[2]))
            lons.append(float(line.split(' ')[3]))
            magnitudes.append(float(line.split(' ')[4]))
    return datestrings, timestrings, lats, lons, magnitudes


   




 


def get_marker_color(magnitude):
    if np.abs(magnitude) < 200.0:
        return ('go')
    elif np.abs(magnitude) < 400.0:         
        return ('yo')     
    else:         
        return ('ro') 
    


        

def plotvaisala (datestrings, timestrings, lats, lons, magnitudes, tbeg, tend, filtag):
    from mpl_toolkits.basemap import Basemap
    import matplotlib.pyplot as plt
    import numpy as np
    
    npoints=93
    import pytz
    utc_dt = datestrings[0].astimezone (pytz.utc)
    firstflashtimestr=utc_dt.isoformat()
    mydate=getantimeridiancrossingtime2(firstflashtimestr)
    utc_dt2 = mydate.astimezone (pytz.utc)

    mydatestr=utc_dt2.isoformat()
    
    lonarr, latarr=getorbit(mydatestr, npoints)
    print ("Creating plot of ground lightning data ... please wait")
    fig=plt.figure( figsize=(15, 6))
    ax = fig.add_axes([0.1,0.1,0.8,0.8])

    minlat= (min (lats))
    maxlat= (max (lats))
    minlon= (min (lons))
    maxlon= (max (lons))
    
    
    minlat= -60
    maxlat= 60
    minlon= -179
    maxlon= 179
    
    minmag=(min (magnitudes))
    maxmag=(max (magnitudes))     
    # --- Build Map ---


    map =  Basemap(projection='merc',llcrnrlat=minlat,urcrnrlat=maxlat,\
                llcrnrlon=minlon,urcrnrlon=maxlon,lat_ts=20,resolution='c')
    map.drawcoastlines()
    map.drawcountries()
    #map.fillcontinents(color = 'gray')
    map.drawmapboundary()
    map.drawmeridians(np.arange(0, 360, 30),labels=[0,0,0,1],fontsize=10)
    map.drawparallels(np.arange(-90, 90, 30),labels=[1,0,0,0],fontsize=10)
    date = datetime.utcnow()
    date=datestrings[0]

    import pytz
    utc_dt = date.astimezone (pytz.utc)
    utc_dt = utc_dt.replace(tzinfo=None)
    CS=map.nightshade(utc_dt)

    min_marker_size = .09
    for lon, lat, mag in zip(lons, lats, magnitudes):
        x,y = map(lon, lat)
        msize = mag * min_marker_size
        marker_string = get_marker_color(mag)
        map.plot(x, y, marker_string, markersize=msize)

    x, y = map(lonarr, latarr) # forgot this line 
    map.plot(x, y, 'D-', markersize=4, linewidth=2, color='k', markerfacecolor='b') 
    map.plot(x, y, '-', linewidth=2, color='k') 
    plt.title("Vaisala GLD 360 data  from "+ tbeg 
              + ' to  '  + tend +' ,  nevents=' + str(len(datestrings)))
    tag = datestrings[0].strftime('%Y%m%dT%H%M%S')
    plt.savefig('vaisala'+filtag+tag+'.png')

    plt.close()

    
def getantimeridiancrossingtime2(timestr):
    import math
    import time
    from datetime import datetime, timedelta
    import ephem
    import numpy as np
    import sys, os
    import pytz
    from dateutil.parser import parse
    
    mydatenow=parse(timestr)
    home = ephem.Observer()
    home.date=mydatenow
    delta_min=timedelta(minutes=1)
    radtodeg=180/np.pi

    lines=gettlefromdb(mydatenow, 1)

    # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
    iss = ephem.readtle(lines[0], lines[1], lines[2])
    
    angle=+178
    angleold=179
    i=0
    while angle < angleold:        
    #for i in range(0,10):
        i=i+1
        mydatenow=parse(timestr)-delta_min*i
        #print (mydatenow)
        angleold=angle+0.0
        home.date=mydatenow
        timeold=mydatenow+timedelta(minutes=1)
        iss.compute(home)        
        angle=iss.sublong*radtodeg
        
        #print mydatenow, angle, angleold
    print ("antimeridian",angleold)
    return (timeold)




def getantimeridiancrossingtime(timestr):
    import math
    import time
    from datetime import datetime, timedelta
    import ephem
    import numpy as np
    import sys, os
    import pytz
    from dateutil.parser import parse
    #mydate=datetime(2016, 4, 7, 9, 10, 45)
    mydatenow=parse(timestr)
    home = ephem.Observer()
    home.date=mydatenow
    lines=gettlefromdb(mydatenow, 1)

    # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
    iss = ephem.readtle(lines[0], lines[1], lines[2])
    degrees_per_radian = 180.0 / math.pi    
    iss.compute(home)

    issorbitsperday=iss._n
    issorbitalperiod=24*60/issorbitsperday


    #print ("ISS sublong" , iss.sublong)
    sublong=iss.sublong.real

    #print ("Position in degrees", sublong*180/np.pi)

    radtodeg=180/np.pi

    angletoam=(sublong+np.pi)
    angletoamdeg=angletoam*radtodeg
    #print ("Angle to AM in degrees", angletoam*radtodeg)
    timetoam=(angletoamdeg)*issorbitalperiod/360


    deltatoam=timedelta(minutes=timetoam)
    mydate=mydatenow-deltatoam


    #print ("Minutes to antimeridian" , timetoam)

    #print ("time Delta to antimeridian" , deltatoam)
    print ("Time ", mydatenow)

    print ("Antimeridian time", mydate)
    home.date = mydate
    iss.compute(home)
            
    print ("antimeridian estimate", iss.sublong*radtodeg)
    return (mydate)



def filtervaisala(datestrings, timestrings, lats, lons, magnitudes):
    # get cached iss tle from database
    lines=gettlefromdb(datestrings[0], 1)
    
    # http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
    import ephem
    import math
    iss = ephem.readtle(lines[0], lines[1], lines[2])

    count=0
    degrees_per_radian = 180.0 / math.pi

    fildate=[]
    filtime=[]
    fillat=[]
    fillon=[]
    filmag=[]
    for i in range (0,datestrings.size):
        lines=gettlefromdb(datestrings[i], 1)
        iss = ephem.readtle(lines[0], lines[1], lines[2])

        iss.compute(datestrings[i])
        distlat=iss.sublat*degrees_per_radian-lats[i]
        distlon=iss.sublong*degrees_per_radian-lons[i]

        asimbox=8

        #print (np.abs(distlat),(np.abs(distlon)) )
        if ( (np.abs(distlat) < asimbox) and (np.abs(distlon) < asimbox)): 
            #print (np.abs(distlat),(np.abs(distlon)) , datestrings[i])
            #print (lats[i], lons[i], iss.sublat, iss.sublong)
            count=count+1

            #print ("yes", i, count)
            filmag.append(magnitudes[i])
            fillon.append(lons[i])
            fillat.append(lats[i])
            fildate.append(datestrings[i])
            filtime.append(timestrings[i])
    print (count)   
    return (fildate, filtime, fillat, fillon, filmag)


def read_postgres_vaisala (tbeg, tend, limit):
    import psycopg2
    import sys
    import numpy as np


    con = None

    try:

        con = psycopg2.connect(host='asdc.space.dtu.dk', database='gld', user='gmurphy', password='j-l%CHB2Za') 
        cur = con.cursor()
        cur.execute("SET TIME ZONE 'UTC';")  # UTC 
        cur.execute("SELECT * FROM lightnings LIMIT 0")
        colnames = [desc[0] for desc in cur.description]
        cur.execute("SELECT * FROM lightnings WHERE ltime   BETWEEN '"+tbeg+"' AND '"+tend+"' ORDER BY ltime ASC, ltime_ns  LIMIT '"+limit+"' ;")          
        data =   (cur.fetchall())

        #for row in rows:
        #    print (row)

    except :
        print ('Error could not connect to database'     )
        sys.exit(1)


    finally:

        if con:
            con.close()


    from numpy import array
    nparr = array( data )
    print ('read_vaisala npar.shape' ,nparr.shape)
    #id=nparr[:,0]
    datestrings=nparr[:,0]
    timestrings=nparr[:,1].astype(int)
    magnitudes=nparr[:,2].astype(float)
    lats=nparr[:,3].astype(float)
    lons=nparr[:,4].astype(float)
    
    return datestrings, timestrings, lats, lons, magnitudes




def perdelta(start, end, delta):
    curr = start
    while curr < end:
        yield curr
        curr += delta

def gettlefromdb(tbeg,  limit):
    import psycopg2
    import sys
    import numpy as np
    from datetime import date, datetime, timedelta
    import pytz
    import ephem
    
    #date_object = datetime.strptime(tbeg, '%Y-%m-%d %H:%M:%S')
    date_object = tbeg
    
    t1=date_object-timedelta(minutes=59)
    t2=date_object+timedelta(minutes=59)
    tbegstr=t1.strftime("%Y-%m-%d %H:%M:%S")
    tendstr=t2.strftime("%Y-%m-%d %H:%M:%S")
    #print (tbegstr, tendstr)
    limitstr=str(limit)
    con = None

    try:

        con = psycopg2.connect(host='asdc.space.dtu.dk', database='gld', user='gmurphy', password='j-l%CHB2Za') 
        cur = con.cursor()
        cur.execute("SET TIME ZONE 'UTC';")  # UTC 
        #colnames = [desc[0] for desc in cur.description]
        cur.execute("SELECT * FROM isstle WHERE logtime   BETWEEN '"+tbegstr+"' AND '"+tendstr+"'   LIMIT '"+limitstr+"' ;")          
        data =   (cur.fetchall())

        #for row in rows:
        #    print (row)

    except :
        print ('Error could not connect to database'     )
        sys.exit(1)


    finally:

        if con:
            con.close()


    from numpy import array
    nparr = array( data )
    #print (data)
    #print ('npar.shape' ,nparr.shape)
    #id=nparr[:,0]

    line0=nparr[0,16]
    line1=nparr[0,17]
    line2=nparr[0,18]
    lines=[line0, line1, line2]
    #print (lines)
    return (lines)

In [4]:



from datetime import date, datetime, timedelta
import pytz
import ephem
#lines=scrapetle()
start_date = datetime(2016, 10, 10, 12, 55, 27,420000, pytz.UTC)

lines=gettlefromdb(start_date,  1)

print (lines)



['ISS (ZARYA)             ', '1 25544U 98067A   16284.17420799  .00006778  00000-0  10957-3 0  9993', '2 25544  51.6448 204.4850 0006808  61.5510  95.6847 15.54113582 22876']


In [ ]:
from __future__ import print_function
   
%matplotlib inline

from datetime import date, datetime, timedelta
import pytz
import ephem
#lines=scrapetle()
start_date = datetime(2016, 10, 3, 12, 13, 0,420000, pytz.UTC)

lines=gettlefromdb(start_date,  1)


#print (lines[0]) 
# http://spaceflight.nasa.gov/realdata/sightings/SSapplications/Post/JavaSSOP/orbit/ISS/SVPOST.html
iss = ephem.readtle(lines[0], lines[1], lines[2])
issorbitinminutes=24*60/iss._n
delta=timedelta(minutes=issorbitinminutes)                
end_date   = start_date+2*delta
for single_date in perdelta(start_date, end_date, delta):
    #
    a=datetime.utcnow()
    mydate=(single_date.strftime("%Y-%m-%d %t"))
    tbeg=single_date
    tend=tbeg+timedelta(minutes=issorbitinminutes)
    tbegstr=tbeg.strftime("%Y-%m-%d %H:%M:%S")
    tendstr=tend.strftime("%Y-%m-%d %H:%M:%S")
    print ("Fetching data from asdc", str(tbeg))
    #
    limitstr=str(4000)
    datestrings, timestrings, lats, lons, magnitudes=read_postgres_vaisala(tbegstr, tendstr, limitstr)
    #datestrings[0]=datestrings[0].astimezone (pytz.utc)
    b=datetime.utcnow()
    print ("Time to fetch",str( b-a))    
    plotvaisala (datestrings, timestrings, lats, lons, magnitudes, tbegstr, tendstr, 'nofil')
    limitstr=str(800000)
    datestrings, timestrings, lats, lons, magnitudes=read_postgres_vaisala(tbegstr, tendstr, limitstr)
    datestrings, timestrings, lats, lons, magnitudes=filtervaisala (datestrings, timestrings, lats, lons, magnitudes)
    nevents=len(datestrings)
    if nevents>0:
        plotvaisala (datestrings, timestrings, lats, lons, magnitudes,tbegstr, tendstr, 'fil')
    else:
        print ("No data available for specified interval")
    c=datetime.utcnow()
    print ("Time to plot",str (c-b))
    
    
    
    
print ("Finished")